In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import os

In [2]:
Input_Filepath = 'D:\\Datasets\\IPL_2024\\RCB VS CSK'

OutputFilePath = 'D:\Datasets\IPL_2024\Data Files'

In [5]:
# Define Data

pattern = r'<p class="ci-html-content">(.*?)<\/p>' # comment

pattern_span = r'<span class="ds-text-tight-s ds-font-regular ds-mb-1 lg:ds-mb-0 lg:ds-mr-3 ds-block ds-text-center ds-text-typo-mid1">(.*?)<\/span>'
 # Over ball

pattern_div = r'<div class="ds-leading-none ds-mb-0.5"><span>(.*?)<\/span><\/div>' # runs and bolwer and batter name

pattern_div_new = r'<div class="[^"]*">(<span>[^<]*</span>)</div>'

def getDatafromHtml(soupe):
    overs = []
    ball_details = []
    comem = []
    batter_Bolwer = []
    text = soupe.find_all('div',class_='lg:hover:ds-bg-ui-fill-translucent ds-hover-parent ds-relative')
    for x in text:
        try:
            overs.append(re.search(pattern_span, str(x), re.DOTALL).group(1).strip())
            ball_details.append(re.search(pattern_div_new, str(x), re.DOTALL).groups())
            comem.append(re.search(pattern, str(x), re.DOTALL).group(1).strip())
            batter_Bolwer.append(re.search(pattern_div, str(x), re.DOTALL).group(1).strip())
        except Exception as e:
            comem.append("No commentry")
    
    data = {
    'overs': overs,
    'ball_details': ball_details,
    'comem': comem,
    'Batter_Bolwer' : batter_Bolwer
    }
    
    return pd.DataFrame(data)

def parse_html(filename, encoding='utf-8'):
    with open(filename, 'r', encoding=encoding) as f:
        soup = BeautifulSoup(f, 'html.parser')
    return soup

def IncludeInng(data, inngs,venue):
    if inngs == 1:
        return data.assign(innings=inngs,venue=venue)
    else:
        return data.assign(innings=inngs,venue=venue)
    
    
def getData(filename,inngs,venu):
    data = getDatafromHtml(parse_html(filename))
    return IncludeInng(data,inngs,venu)


def getMatchData(filename1,filename2,venue):
    df = pd.concat(
        [
            getData(filename1,1,venue),
            getData(filename2,2,venue)
        ],
        ignore_index=True
    )
    return df


def getfiles():
    os.chdir(Input_Filepath)
    return [file for file in os.listdir() if file.endswith(".html")]


def WriteDataToFile(File1,File2,venue,filename):
    os.chdir(Input_Filepath)
    getMatchData(File1,File2,venue).to_csv(os.path.join(OutputFilePath, filename),index=False)

In [6]:
getfiles()

['First_Innings_RCB.html', 'Second_Innings_CSK_Files.html']

In [ ]:
""""
soup = parse_html('First_Innings_RCB.html')

text = soup.find_all('div',class_='lg:hover:ds-bg-ui-fill-translucent ds-hover-parent ds-relative')

for x in text:
    print(re.search(pattern_span, str(x), re.DOTALL).group(1).strip())
    print(re.search(pattern_div_new, str(x), re.DOTALL).groups())
    print(re.search(pattern, str(x), re.DOTALL).group(1).strip())
    print(re.search(pattern_div, str(x), re.DOTALL).group(1).strip())
    
""""

In [8]:
WriteDataToFile('First_Innings_RCB.html','Second_Innings_CSK_Files.html','MA Chidambaram Stadium, Chepauk, Chennai','RCB_vs_CSK.csv')